<b>Deep Learning</b>

In [1]:

from typing import List
Tensor = List
def shape(tensor:Tensor) -> List[int]:
    sizes: List[int] = []
    while isinstance(tensor,list):
        sizes.append(len(tensor))
        tensor = tensor[0]
    return sizes
assert shape([1,2,3]) == [3]

In [2]:
def is_1d(tensor:Tensor) -> bool:
    return not isinstance(tensor[0],list)

assert is_1d([1,2,3])

In [3]:
def tensor_sum(tensor:Tensor) -> float:
    if is_1d(tensor):
        return sum(tensor)
    else:
        return sum(tensor_sum(tensor_i)
                  for tensor_i in tensor)


assert tensor_sum([1,2,3]) == 6

In [4]:
from typing import Callable

def tensor_apply(f:Callable[[float],float],tensor:Tensor) -> Tensor:
    if is_1d(tensor):
        return [f(x) for x in tensor]
    else:
        return [tensor_apply(f,tensor_i) for tensor_i in tensor]

assert tensor_apply(lambda x: x + 1,[1,2,3]) == [2,3,4]

In [5]:
def zeros_like(tensor:Tensor) -> Tensor:
    return tensor_apply(lambda _: 0.0,tensor)

assert zeros_like([1,2,3]) == [0,0,0]

In [6]:
def tensor_combine(f:Callable[[float,],float],
                  t1:Tensor,
                  t2:Tensor) -> Tensor:
    if is_1d(t1):
        return [f(x,y) for x,y in zip(t1,t2)]
    else:
        return [tensor_combine(f,t1_i,t2_i)
               for t1_i,t2_i in zip(t1,t2)]

import operator
assert tensor_combine(operator.add,[1,2,3],[4,5,6]) == [5,7,9]

In [7]:
from typing import Iterable,Tuple
class Layer:
    
    def forward(self,input):
        raise NotImplementedError
    
    def backward(self,gradient):
        raise NotImplementedError
    
    def params(self) -> Iterable[Tensor]:
        return ()

In [8]:
from scratch.neural_networks import sigmoid

class Sigmoid(Layer):
    def forward(self,input:Tensor) -> Tensor:
        self.sigmoid = tensor_apply(sigmoid,input)
        return self.sigmoids
    
    def backward(self,gradient:Tensor) -> Tensor:
        return tensor_combine(lambda sig,grad:sig*(1-sig) * grad,
                              self.sigmoids,
                              gradient)

In [9]:
import random
from scratch.probability import inverse_normal_cdf

<Figure size 432x288 with 0 Axes>

In [10]:
def random_uniform(*dims:int) -> Tensor:
    if len(dims) == 1:
        return [random.random() for _ in range(dims[0])]
    else:
        return [random_uniform(*dims[1:]) for _ in range(dims[0])]

def random_normal(*dims:int,
                 mean: float = 0.0,
                 varience: float=1.0) -> Tensor:
    if len(dims) == 1:
        return [mean+varience*inverse_normal_cdf(random.random())
               for _ in range(dims[0])]
    
assert shape(random_uniform(2,3,4)) == [2,3,4]

In [11]:
def random_tensor(*dims:int,init:str='normal') -> Tensor:
    if init == 'normal':
        return random_normal(*dims)
    elif init == 'uniform':
        return random_uniform(*dims)
    elif init == 'xavier':
        varience = len(dims) / sum(dims)
        return random_normal(*dims,varience=varience)
    else:
        raise ValueError(f"unknown init:{init}")

In [12]:
from scratch.linear_algebra import dot

class Linear(Layer):
    def __init__(self,
                input_dim:int,
                output_dim:int,
                init: str='xavier')-> None:
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.w = random_tensor(output_dim,input_dim,init=init)
        self.b = random_tensor(output_dim,init=init)
        
    def forward(self,input:Tensor) -> Tensor:
        self.input = input
        return  [dot(input, self.w[o] + f.b[o])
                for o in range(self.output_dim)]
    
    def backward(self,gradient:Tensor) -> Tensor:
        self.b_grad = gradient
        self.w_grad = [[self.input[i] * gradient[o]
                       for i in range(self.input_dim)]
                      for o in range(self.output_dim)]
        return [sum(self.w[o][i] * gradient[o] for o in range(self.output_dim)) for i in range(self.input_dim)]
  
    def params(self) -> Iterable[Tensor]:
        return [self.w,self.b]
    
    def grads(self) -> Iterable[Tensor]:
        return [self.w_grad,self.b_grad]

In [13]:
from typing import List

class Sequential(Layer):
    def __init__(self,layers:List[Layer]) -> None:
        self.layers = layers
    def forward(self,input):
        for layer in self.layers:
            input = layer.forward(input)
        return input
    def backward(self,gradient):
        for layer in reversed(self.layers):
            gradient = layer.backward(gradient)
        return gradient
    def params(self) -> Iterable[Tensor]:
        return (param for layer in self.layers for param in layer.params())
    def grads(self) -> Iterable[Tensor]:
        return (grad  for layer in self.layers for grad in layer.grads())

In [14]:
xor_net = Sequential([
    Linear(input_dim=2,output_dim=2),
    Sigmoid(),
    Linear(input_dim=2,output_dim=2),
    Sigmoid()
])

In [15]:
class Loss:
    def loss(self,predicted:Tensor,actual:Tensor) -> float:
        raise NotImplementedError
    def gradient(self,predicted:Tensor,actual:Tensor) -> Tensor:
        raise NotImplementedError

In [16]:
class SSE(Loss):
    def loss(self,predicted:Tensor,actual:Tensor) -> float:
        squared_errors = tensor_combine(
        lambda predicted,actual:(predicted - actual) ** 2,
            predicted,
            actual)
        return tensor_sum(squared_errors)
    
    def gradient(self,predicted:Tensor,actual:Tensor) -> Tensor:
        return tensor_combine(
        lambda predicted,actual:2*(predicted - actual),
        predicted,
        actual)

In [18]:
theta = gradient_step(theta,grad,-learning_rate)

NameError: name 'gradient_step' is not defined

In [19]:
class Optimizer:
    def step(self,layer:Layer) -> None:
        raise NotImplementedError

class GradientDescent(Optimizer):
    def __init__(self,learning_rate:float=0.1)->None:
        self.lr = learning_rate
    
    def step(self,layer:layer) -> None:
        for param, grad in zip(layer.params(),layer.grads()):
            param[:] = tensor_combine(
                lambda param,grad: param - grad * self.lr,
                param,
                grad)

NameError: name 'Optimizer' is not defined